# Lab 3.6.1: Building Custom Tools for AI Agents

**Module:** 3.6 - AI Agents & Agentic Systems  
**Time:** 2 hours  
**Difficulty:** ⭐⭐⭐ (Intermediate)

---

## 🎯 Learning Objectives

By the end of this notebook, you will:
- [ ] Understand what tools are and why agents need them
- [ ] Create 5 production-ready custom tools
- [ ] Implement safe code execution and file operations
- [ ] Build tools that integrate with external APIs
- [ ] Handle tool errors gracefully

---

## 📚 Prerequisites

- Completed: Module 3.5 (RAG Systems)
- Knowledge of: Python functions, decorators, error handling
- Running: Ollama with `llama3.1:8b` model

---

## 🌍 Real-World Context

**The Problem:** LLMs can only generate text - they can't actually DO things.

**Examples of LLM Limitations:**
- 🔢 "Calculate 15% tip on $127.50" → LLMs often make arithmetic errors
- 🌐 "What's the weather in Tokyo?" → LLMs don't have real-time data
- 📁 "Read my config.json file" → LLMs can't access your filesystem
- 💻 "Run this Python code" → LLMs can't execute code

**Tools solve this** by giving agents the ability to perform actions and get real results!

**Real-World Applications:**
- **Customer Support Bots**: Look up orders, process refunds, update accounts
- **Coding Assistants**: Run tests, check syntax, search documentation
- **Research Agents**: Search papers, download datasets, run analyses
- **Personal Assistants**: Check calendars, send emails, book appointments

---

## 🧒 ELI5: What Are Agent Tools?

> **Imagine a super-smart friend who knows everything BUT has no hands...** 🤔
>
> They can tell you HOW to bake a cake, but can't actually mix the ingredients.
> They can explain HOW to fix a car, but can't hold a wrench.
>
> **Tools are like giving that friend hands!**
>
> - "I need to calculate 15% of 127.50" → 🧮 Uses calculator tool → Gets exact answer
> - "What's in that file?" → 📄 Uses file reader tool → Sees actual contents
> - "Search for latest news" → 🔍 Uses web search tool → Gets real results
>
> **In AI terms:**
> - The LLM decides WHAT tool to use and WITH what inputs
> - The tool EXECUTES the action and returns results
> - The LLM interprets the results and continues reasoning

```
┌─────────────────────────────────────────────────────────────┐
│                    Agent Tool Loop                          │
├─────────────────────────────────────────────────────────────┤
│                                                             │
│  User: "What's 15% tip on $127.50 bill?"                   │
│              │                                              │
│              ▼                                              │
│     ┌────────────────┐                                      │
│     │  LLM Thinks:   │                                      │
│     │ "I need to     │                                      │
│     │  calculate..." │                                      │
│     └───────┬────────┘                                      │
│             │                                               │
│             ▼                                               │
│  ┌──────────────────────────────────┐                       │
│  │ Tool Selection:                  │                       │
│  │   🧮 Calculator                  │ ← Decides which tool  │
│  │   Input: "127.50 * 0.15"         │ ← Formats input       │
│  └──────────────┬───────────────────┘                       │
│                 │                                           │
│                 ▼                                           │
│  ┌──────────────────────────────────┐                       │
│  │ Tool Execution:                  │                       │
│  │   Result: 19.125                 │ ← Real calculation!   │
│  └──────────────┬───────────────────┘                       │
│                 │                                           │
│                 ▼                                           │
│     ┌────────────────┐                                      │
│     │ LLM Response:  │                                      │
│     │ "The 15% tip   │                                      │
│     │ is $19.13"     │                                      │
│     └────────────────┘                                      │
│                                                             │
└─────────────────────────────────────────────────────────────┘
```

---

## Part 1: Environment Setup

In [ ]:
# Standard imports
import os
import sys
from pathlib import Path
from typing import Optional, Dict, Any, List
import json
import time
import ast
import operator
import math
import re
import io
import contextlib
import urllib.request
import urllib.parse
import urllib.error
from datetime import datetime

# Add scripts to path
sys.path.insert(0, str(Path.cwd().parent / 'scripts'))

print("✅ Standard imports successful!")

In [ ]:
# LangChain imports
try:
    from langchain.tools import tool, Tool, StructuredTool
    from langchain_community.llms import Ollama
    print("✅ LangChain imports successful!")
except ImportError as e:
    print(f"❌ LangChain not installed. Run: pip install langchain langchain-community")
    print(f"   Error: {e}")

In [ ]:
# Verify Ollama is running
import requests

def check_ollama():
    """Check if Ollama is running and has required models."""
    try:
        resp = requests.get("http://localhost:11434/api/tags", timeout=5)
        if resp.status_code == 200:
            models = [m['name'] for m in resp.json().get('models', [])]
            print(f"✅ Ollama is running!")
            print(f"   Available models: {', '.join(models[:5])}")
            
            # Check for recommended model
            if any('llama3' in m for m in models):
                print("   ✅ Llama 3 model available")
            else:
                print("   ⚠️ Llama 3 not found. Run: ollama pull llama3.1:8b")
            return True
    except:
        pass
    
    print("❌ Ollama not running!")
    print("   Start with: ollama serve")
    print("   Then pull a model: ollama pull llama3.1:8b")
    return False

check_ollama()

---

## Part 2: Tool 1 - Safe Calculator

Our first tool is a calculator that can evaluate mathematical expressions **safely** - without using dangerous `eval()`!

### Why Not Just Use `eval()`?

```python
# ❌ DANGEROUS - Never do this!
eval("__import__('os').system('rm -rf /')")
```

An attacker could execute arbitrary code! Instead, we'll parse expressions safely using Python's AST (Abstract Syntax Tree).

In [ ]:
class SafeCalculator:
    """
    A safe mathematical expression evaluator.
    
    Unlike using eval(), this parser only allows mathematical operations,
    making it safe from code injection attacks.
    
    Supported operations:
        - Basic arithmetic: +, -, *, /, //, %, **
        - Functions: sin, cos, tan, sqrt, log, log10, exp, abs, round, floor, ceil
        - Constants: pi, e
    
    Example:
        >>> calc = SafeCalculator()
        >>> result = calc.evaluate("sqrt(16) + 2**3")
        >>> print(result)
        12.0
    """
    
    # Map AST operators to Python functions
    OPERATORS = {
        ast.Add: operator.add,
        ast.Sub: operator.sub,
        ast.Mult: operator.mul,
        ast.Div: operator.truediv,
        ast.FloorDiv: operator.floordiv,
        ast.Mod: operator.mod,
        ast.Pow: operator.pow,
        ast.USub: operator.neg,
        ast.UAdd: operator.pos,
    }
    
    # Allowed mathematical functions
    FUNCTIONS = {
        'sin': math.sin,
        'cos': math.cos,
        'tan': math.tan,
        'asin': math.asin,
        'acos': math.acos,
        'atan': math.atan,
        'sqrt': math.sqrt,
        'log': math.log,
        'log10': math.log10,
        'log2': math.log2,
        'exp': math.exp,
        'abs': abs,
        'round': round,
        'floor': math.floor,
        'ceil': math.ceil,
        'factorial': math.factorial,
    }
    
    # Mathematical constants
    CONSTANTS = {
        'pi': math.pi,
        'e': math.e,
        'tau': math.tau,
    }
    
    def evaluate(self, expression: str) -> float:
        """Safely evaluate a mathematical expression."""
        expression = expression.strip()
        
        try:
            # Parse into AST
            tree = ast.parse(expression, mode='eval')
            return self._eval_node(tree.body)
        except Exception as e:
            raise ValueError(f"Invalid expression: {e}")
    
    def _eval_node(self, node) -> float:
        """Recursively evaluate an AST node."""
        # Handle numeric constants
        if isinstance(node, ast.Constant):
            if isinstance(node.value, (int, float)):
                return node.value
            raise ValueError(f"Unsupported constant type: {type(node.value)}")
        
        # Handle named constants (pi, e, etc.)
        elif isinstance(node, ast.Name):
            if node.id in self.CONSTANTS:
                return self.CONSTANTS[node.id]
            raise ValueError(f"Unknown constant: {node.id}")
        
        # Handle binary operations (+, -, *, /, etc.)
        elif isinstance(node, ast.BinOp):
            left = self._eval_node(node.left)
            right = self._eval_node(node.right)
            op_type = type(node.op)
            
            if op_type not in self.OPERATORS:
                raise ValueError(f"Unsupported operator: {op_type.__name__}")
            
            return self.OPERATORS[op_type](left, right)
        
        # Handle unary operations (-x, +x)
        elif isinstance(node, ast.UnaryOp):
            operand = self._eval_node(node.operand)
            op_type = type(node.op)
            
            if op_type not in self.OPERATORS:
                raise ValueError(f"Unsupported operator: {op_type.__name__}")
            
            return self.OPERATORS[op_type](operand)
        
        # Handle function calls (sin, sqrt, etc.)
        elif isinstance(node, ast.Call):
            func_name = node.func.id if isinstance(node.func, ast.Name) else None
            
            if func_name not in self.FUNCTIONS:
                raise ValueError(f"Unknown function: {func_name}")
            
            args = [self._eval_node(arg) for arg in node.args]
            return self.FUNCTIONS[func_name](*args)
        
        else:
            raise ValueError(f"Unsupported expression type: {type(node).__name__}")


# Test the calculator
print("🧪 Testing SafeCalculator:")
calc = SafeCalculator()

tests = [
    ("2 + 2", 4),
    ("127.50 * 0.15", 19.125),
    ("sqrt(16) + 2**3", 12.0),
    ("sin(pi/2)", 1.0),
    ("log10(100)", 2.0),
]

for expr, expected in tests:
    result = calc.evaluate(expr)
    status = "✅" if abs(result - expected) < 0.0001 else "❌"
    print(f"  {status} {expr} = {result} (expected: {expected})")

In [ ]:
# Now wrap it as a LangChain tool
@tool
def calculate(expression: str) -> str:
    """
    Safely evaluate a mathematical expression.
    
    Use this tool when you need to perform calculations. Supports:
    - Basic arithmetic: +, -, *, /, //, %, **
    - Functions: sin, cos, tan, sqrt, log, exp, abs, round, floor, ceil
    - Constants: pi, e
    
    Args:
        expression: A mathematical expression like "sqrt(16) + 2**3"
    
    Returns:
        The result as a string
    
    Examples:
        calculate("2 + 2") -> "4"
        calculate("sqrt(16) * 3") -> "12"
        calculate("sin(pi/2)") -> "1"
    """
    try:
        calc = SafeCalculator()
        result = calc.evaluate(expression)
        
        # Format nicely
        if isinstance(result, float) and result.is_integer():
            return f"{expression} = {int(result)}"
        return f"{expression} = {result:.10g}"
    except Exception as e:
        return f"Error: {str(e)}"


# Test the tool
print("🧪 Testing Calculator Tool:")
print(f"  {calculate.invoke('15 + 27')}")
print(f"  {calculate.invoke('127.50 * 0.15')}")
print(f"  {calculate.invoke('sqrt(144) + 5')}")

### 🔍 What Just Happened?

We created a **safe** calculator that:
1. Uses Python's AST (Abstract Syntax Tree) to parse expressions
2. Only allows whitelisted operations and functions
3. Prevents code injection (unlike `eval()`!)
4. Returns helpful error messages

**Security Note:** Never use `eval()` in production code! It can execute arbitrary Python code.

---

## Part 3: Tool 2 - Web Search (Simulated)

In production, you'd connect to a real search API (Google, Bing, SerpAPI). Here's a simulated version for learning that demonstrates the pattern.

In [ ]:
@tool
def web_search(query: str, num_results: int = 5) -> str:
    """
    Search the web for information.
    
    Use this tool when you need to find current information from the internet.
    
    Args:
        query: The search query
        num_results: Number of results to return (default: 5)
    
    Returns:
        Search results as formatted text
    
    Note:
        This is a simulated search for educational purposes.
        In production, integrate with a real search API like:
        - Google Custom Search API
        - Bing Search API  
        - SerpAPI
        - Tavily Search
    """
    # Simulated knowledge base for demo
    simulated_results = {
        "dgx spark": [
            {
                "title": "NVIDIA DGX Spark Specifications",
                "url": "https://www.nvidia.com/dgx-spark",
                "snippet": "DGX Spark features the NVIDIA Blackwell GB10 Superchip with 128GB unified LPDDR5X memory, 6,144 CUDA cores, and 192 5th-gen Tensor Cores."
            },
            {
                "title": "DGX Spark: Desktop AI Supercomputer",
                "url": "https://www.nvidia.com/blog/dgx-spark-desktop-ai",
                "snippet": "The DGX Spark delivers 1 PFLOP of FP4 compute power in a desktop form factor, enabling local development of large language models."
            }
        ],
        "langchain": [
            {
                "title": "LangChain Documentation",
                "url": "https://python.langchain.com/docs/",
                "snippet": "LangChain is a framework for developing applications powered by language models through composable components."
            }
        ],
        "llama 3": [
            {
                "title": "Meta Llama 3.1 Model Card",
                "url": "https://huggingface.co/meta-llama/Llama-3.1",
                "snippet": "Llama 3.1 is Meta's latest open-source LLM, available in 8B, 70B, and 405B parameter versions with 128K context length."
            }
        ],
        "react pattern": [
            {
                "title": "ReAct: Synergizing Reasoning and Acting in LLMs",
                "url": "https://arxiv.org/abs/2210.03629",
                "snippet": "ReAct prompts LLMs to generate verbal reasoning traces and task-specific actions in an interleaved manner."
            }
        ],
        "langgraph": [
            {
                "title": "LangGraph Documentation",
                "url": "https://langchain-ai.github.io/langgraph/",
                "snippet": "LangGraph is a library for building stateful, multi-actor applications with LLMs using graph-based orchestration."
            }
        ]
    }
    
    # Search for matching results
    query_lower = query.lower()
    results = []
    
    for key, items in simulated_results.items():
        if any(word in query_lower for word in key.split()):
            results.extend(items)
    
    if not results:
        # Default response for unknown queries
        return f"No results found for: {query}\n\nNote: This is a simulated search. In production, connect to a real search API."
    
    # Format results
    output = f"Search results for: {query}\n\n"
    for i, result in enumerate(results[:num_results], 1):
        output += f"{i}. {result['title']}\n"
        output += f"   URL: {result['url']}\n"
        output += f"   {result['snippet']}\n\n"
    
    return output


# Test the search tool
print("🧪 Testing Web Search Tool:")
print(web_search.invoke("DGX Spark specs"))

---

## Part 4: Tool 3 - Code Execution (Sandboxed)

⚠️ **Security Warning:** Running arbitrary code is dangerous! This is a heavily restricted version for educational purposes.

### Production Security Measures

In production, you MUST use proper sandboxing:
- Docker containers with resource limits
- gVisor or Firecracker for isolation
- AWS Lambda or Cloud Functions
- Dedicated services like Judge0 or Piston

In [ ]:
class SafePythonExecutor:
    """
    Execute Python code in a restricted environment.
    
    SECURITY WARNING - EDUCATIONAL USE ONLY:
    This is NOT suitable for untrusted code execution.
    In production, use proper sandboxing (Docker, gVisor, etc.)
    """
    
    # Patterns that are FORBIDDEN
    FORBIDDEN_PATTERNS = [
        r'\bimport\s+os\b',
        r'\bimport\s+sys\b', 
        r'\bimport\s+subprocess\b',
        r'\bimport\s+socket\b',
        r'\bimport\s+requests\b',
        r'\bimport\s+urllib\b',
        r'\bimport\s+shutil\b',
        r'\bopen\s*\(',
        r'\bexec\s*\(',
        r'\beval\s*\(',
        r'__builtins__',
        r'__import__',
        r'\bcompile\s*\(',
        r'\bglobals\s*\(',
        r'\blocals\s*\(',
    ]
    
    def __init__(self, timeout: int = 5):
        self.timeout = timeout
    
    def _check_safety(self, code: str) -> tuple:
        """Check if code contains forbidden patterns."""
        for pattern in self.FORBIDDEN_PATTERNS:
            if re.search(pattern, code, re.IGNORECASE):
                return False, f"Forbidden pattern detected: {pattern}"
        return True, None
    
    def execute(self, code: str) -> dict:
        """Execute code safely and return the result."""
        # Check for forbidden patterns
        is_safe, error = self._check_safety(code)
        if not is_safe:
            return {
                'output': '',
                'error': error,
                'success': False
            }
        
        try:
            # Create restricted globals
            restricted_globals = {
                '__builtins__': {
                    'print': print,
                    'len': len,
                    'range': range,
                    'enumerate': enumerate,
                    'zip': zip,
                    'map': map,
                    'filter': filter,
                    'sum': sum,
                    'min': min,
                    'max': max,
                    'abs': abs,
                    'round': round,
                    'sorted': sorted,
                    'reversed': reversed,
                    'list': list,
                    'dict': dict,
                    'set': set,
                    'tuple': tuple,
                    'str': str,
                    'int': int,
                    'float': float,
                    'bool': bool,
                    'type': type,
                    'isinstance': isinstance,
                    'True': True,
                    'False': False,
                    'None': None,
                },
                'math': math,
            }
            
            # Also allow common safe imports within the code
            import random
            import json as json_mod
            from datetime import datetime as dt, timedelta
            from collections import Counter, defaultdict
            
            restricted_globals['random'] = random
            restricted_globals['json'] = json_mod
            restricted_globals['datetime'] = dt
            restricted_globals['timedelta'] = timedelta
            restricted_globals['Counter'] = Counter
            restricted_globals['defaultdict'] = defaultdict
            
            # Capture output
            output_buffer = io.StringIO()
            with contextlib.redirect_stdout(output_buffer):
                exec(code, restricted_globals)
            
            output = output_buffer.getvalue().strip()
            return {
                'output': output if output else "Code executed successfully (no output)",
                'error': None,
                'success': True
            }
        
        except Exception as e:
            return {
                'output': '',
                'error': f"{type(e).__name__}: {str(e)}",
                'success': False
            }


@tool
def execute_python(code: str) -> str:
    """
    Execute Python code in a restricted environment.
    
    Use this when you need to run Python code to solve a problem.
    Available: math, random, json, datetime, Counter.
    NOT available: file operations, network, system commands.
    
    Args:
        code: Python code to execute
    
    Returns:
        The printed output of the code execution
    
    Examples:
        execute_python("print(2 ** 10)") -> "1024"
        execute_python("print([i**2 for i in range(5)])") -> "[0, 1, 4, 9, 16]"
    """
    executor = SafePythonExecutor()
    result = executor.execute(code)
    
    if result['success']:
        return result['output']
    else:
        return f"Error: {result['error']}"


# Test the code executor
print("🧪 Testing Code Executor Tool:")

# Good examples
print("\n✅ Valid code examples:")
print(f"  Squares: {execute_python.invoke('print([i**2 for i in range(5)])')}")
print(f"  Factorial: {execute_python.invoke('print(math.factorial(10))')}")
print(f"  Random: {execute_python.invoke('print(random.randint(1, 100))')}")

# Forbidden examples  
print("\n❌ Forbidden code (blocked):")
print(f"  File read: {execute_python.invoke('import os; print(os.getcwd())')}")
print(f"  Eval: {execute_python.invoke('eval(\"1+1\")')}")

---

## Part 5: Tool 4 - File Reader

A tool that can read files safely with proper validation.

In [ ]:
@tool
def read_file(file_path: str, max_chars: int = 10000) -> str:
    """
    Read the contents of a text file.
    
    Use this tool when you need to read a file from the filesystem.
    
    Args:
        file_path: Path to the file to read
        max_chars: Maximum characters to return (default: 10000)
    
    Returns:
        The file contents or an error message
    
    Note:
        For security, only reads text files with allowed extensions
        and has a size limit.
    """
    try:
        path = Path(file_path).resolve()
        
        # Security: Check if file exists
        if not path.exists():
            return f"Error: File not found: {file_path}"
        
        if not path.is_file():
            return f"Error: Not a file: {file_path}"
        
        # Security: Check file size (limit to 1MB)
        file_size = path.stat().st_size
        if file_size > 1_000_000:
            return f"Error: File too large ({file_size:,} bytes). Maximum is 1MB."
        
        # Security: Only allow text files
        allowed_extensions = {
            '.txt', '.md', '.json', '.py', '.csv', 
            '.yaml', '.yml', '.xml', '.html', '.css', 
            '.js', '.ts', '.sh', '.bash', '.toml'
        }
        
        if path.suffix.lower() not in allowed_extensions:
            return f"Error: Unsupported file type: {path.suffix}. Allowed: {', '.join(sorted(allowed_extensions))}"
        
        # Read the file
        with open(path, 'r', encoding='utf-8') as f:
            content = f.read(max_chars)
        
        if len(content) == max_chars:
            content += f"\n\n... (truncated at {max_chars:,} characters)"
        
        return content
    
    except UnicodeDecodeError:
        return "Error: File is not a text file or uses unsupported encoding"
    except PermissionError:
        return f"Error: Permission denied for file: {file_path}"
    except Exception as e:
        return f"Error reading file: {str(e)}"


# Test the file reader
print("🧪 Testing File Reader Tool:")

# Test with README
readme_path = Path.cwd().parent / "README.md"
if readme_path.exists():
    result = read_file.invoke(str(readme_path))
    print(f"\n📄 README.md (first 500 chars):")
    print(result[:500])
else:
    print(f"\n⚠️ README.md not found at {readme_path}")

# Test error cases
print(f"\n❌ Non-existent file: {read_file.invoke('/nonexistent/file.txt')}")
print(f"❌ Binary file: {read_file.invoke('/bin/ls')}")

---

## Part 6: Tool 5 - API Caller

A tool for making HTTP requests to external APIs with security checks.

In [ ]:
@tool
def call_api(
    url: str,
    method: str = "GET",
    headers: str = "",
    body: str = ""
) -> str:
    """
    Make an HTTP API call.
    
    Use this tool when you need to fetch data from a web API.
    
    Args:
        url: The API endpoint URL (must be HTTPS for security)
        method: HTTP method - GET or POST (default: GET)
        headers: Optional JSON string of headers
        body: Optional request body for POST requests
    
    Returns:
        The API response or an error message
    
    Examples:
        call_api("https://api.github.com/users/octocat")
        call_api("https://httpbin.org/post", method="POST", body='{"key": "value"}')
    """
    try:
        # Validate URL
        parsed = urllib.parse.urlparse(url)
        
        if parsed.scheme not in ('http', 'https'):
            return "Error: Only HTTP/HTTPS URLs are allowed"
        
        # Security: Block internal/local URLs
        blocked_hosts = [
            'localhost', '127.0.0.1', '0.0.0.0', 
            '169.254.169.254',  # AWS metadata
            '10.', '172.16.', '192.168.'  # Private ranges
        ]
        
        hostname = parsed.hostname or ''
        for blocked in blocked_hosts:
            if hostname.startswith(blocked) or hostname == blocked.rstrip('.'):
                return f"Error: Access to internal addresses ({hostname}) is not allowed"
        
        # Parse headers if provided
        header_dict = {}
        if headers:
            try:
                header_dict = json.loads(headers)
            except json.JSONDecodeError:
                return "Error: Invalid headers JSON"
        
        # Add default headers
        header_dict.setdefault('User-Agent', 'DGX-Spark-Agent/1.0')
        header_dict.setdefault('Accept', 'application/json')
        
        # Create and send request
        if method.upper() == "POST":
            data = body.encode('utf-8') if body else None
            header_dict.setdefault('Content-Type', 'application/json')
            req = urllib.request.Request(url, data=data, headers=header_dict, method='POST')
        else:
            req = urllib.request.Request(url, headers=header_dict, method='GET')
        
        # Make request with timeout
        with urllib.request.urlopen(req, timeout=10) as response:
            content = response.read().decode('utf-8')
            
            # Truncate if too long
            max_length = 5000
            if len(content) > max_length:
                content = content[:max_length] + "\n... (truncated)"
            
            return f"Status: {response.status}\n\nResponse:\n{content}"
    
    except urllib.error.HTTPError as e:
        return f"HTTP Error {e.code}: {e.reason}"
    except urllib.error.URLError as e:
        return f"URL Error: {str(e.reason)}"
    except Exception as e:
        return f"Error: {str(e)}"


# Test the API caller
print("🧪 Testing API Caller Tool:")

# Test with a public API
print("\n📡 Fetching from httpbin.org:")
result = call_api.invoke("https://httpbin.org/get")
print(result[:500] + "..." if len(result) > 500 else result)

# Test blocked local access
print(f"\n❌ Blocked: {call_api.invoke('http://localhost:8080')}")

---

## Part 7: Bonus Tools - Datetime and JSON Parser

In [ ]:
@tool
def get_current_datetime(format: str = "%Y-%m-%d %H:%M:%S") -> str:
    """
    Get the current date and time.
    
    Use this tool when you need to know the current date or time.
    
    Args:
        format: strftime format string (default: "%Y-%m-%d %H:%M:%S")
    
    Returns:
        Current datetime as a formatted string
    
    Examples:
        get_current_datetime()  # -> "2024-01-15 14:30:00"
        get_current_datetime("%A, %B %d, %Y")  # -> "Monday, January 15, 2024"
    """
    return datetime.now().strftime(format)


@tool
def json_parser(json_string: str, path: str = "") -> str:
    """
    Parse JSON and optionally extract a specific path.
    
    Use this tool to parse JSON data and extract values.
    
    Args:
        json_string: The JSON string to parse
        path: Optional dot-notation path to extract (e.g., "data.users[0].name")
    
    Returns:
        Parsed JSON or extracted value
    
    Examples:
        json_parser('{"name": "John"}')  # -> formatted JSON
        json_parser('{"data": {"id": 123}}', "data.id")  # -> "123"
    """
    try:
        data = json.loads(json_string)
        
        if not path:
            return json.dumps(data, indent=2)
        
        # Navigate the path
        current = data
        for part in path.replace('[', '.').replace(']', '').split('.'):
            if not part:
                continue
            if isinstance(current, list):
                current = current[int(part)]
            elif isinstance(current, dict):
                current = current[part]
            else:
                return f"Error: Cannot navigate to '{part}' in {type(current).__name__}"
        
        if isinstance(current, (dict, list)):
            return json.dumps(current, indent=2)
        return str(current)
    
    except json.JSONDecodeError as e:
        return f"JSON Parse Error: {e}"
    except (KeyError, IndexError) as e:
        return f"Path Error: {e}"
    except Exception as e:
        return f"Error: {e}"


# Test the bonus tools
print("🧪 Testing Bonus Tools:")

print(f"\n⏰ Current datetime: {get_current_datetime.invoke('')}")
print(f"   Formatted: {get_current_datetime.invoke('%A, %B %d, %Y')}")

sample_json = '{"model": "llama3", "config": {"temperature": 0.7, "max_tokens": 100}}'
print(f"\n📋 JSON parsing:")
print(f"   Full: {json_parser.invoke(sample_json)}")
print(f"   Extract: {json_parser.invoke(sample_json + '|config.temperature').split('|')[0]}")

---

## Part 8: Putting It All Together - Tool Registry

In [ ]:
# Collect all tools
all_tools = [
    calculate,
    web_search,
    execute_python,
    read_file,
    call_api,
    get_current_datetime,
    json_parser,
]


def get_tools_summary():
    """Get a formatted summary of all available tools."""
    print("\n" + "="*60)
    print("🛠️  AVAILABLE TOOLS")
    print("="*60)
    
    for i, tool in enumerate(all_tools, 1):
        name = tool.name
        # Get first line of description
        desc = tool.description.split('\n')[0].strip()
        print(f"\n{i}. {name}")
        print(f"   {desc[:70]}..." if len(desc) > 70 else f"   {desc}")
    
    print("\n" + "="*60)


get_tools_summary()

---

## ⚠️ Common Mistakes

### Mistake 1: Missing or Poor Tool Descriptions

In [ ]:
# ❌ WRONG: No description - agent doesn't know when to use it
# @tool
# def bad_tool(x):
#     return x * 2

# ✅ RIGHT: Clear description with examples
@tool
def double_number(number: str) -> str:
    """
    Double a number.
    
    Use this when you need to multiply a number by 2.
    
    Args:
        number: The number to double (as a string)
    
    Returns:
        The doubled number
    
    Example:
        double_number("5") -> "10"
    """
    try:
        return str(float(number) * 2)
    except ValueError:
        return f"Error: '{number}' is not a valid number"

print("Good descriptions help agents know WHEN and HOW to use tools!")

### Mistake 2: Not Handling Errors

In [ ]:
# ❌ WRONG: Crashes on invalid input
# @tool
# def bad_divide(expression: str) -> str:
#     """Divide two numbers."""
#     a, b = expression.split("/")
#     return str(int(a) / int(b))  # Crashes on "10/0" or "abc/def"

# ✅ RIGHT: Handle errors gracefully
@tool
def safe_divide(expression: str) -> str:
    """
    Divide two numbers. Format: 'a/b'
    
    Args:
        expression: Division expression like "10/2"
    
    Returns:
        The result or an error message
    """
    try:
        parts = expression.split("/")
        if len(parts) != 2:
            return "Error: Use format 'a/b' like '10/2'"
        
        a = float(parts[0].strip())
        b = float(parts[1].strip())
        
        if b == 0:
            return "Error: Cannot divide by zero"
        
        return str(a / b)
    except ValueError:
        return "Error: Both values must be numbers"

print(f"Division by zero: {safe_divide.invoke('10/0')}")
print(f"Invalid input: {safe_divide.invoke('abc/def')}")
print(f"Valid: {safe_divide.invoke('10/2')}")

### Mistake 3: Security Vulnerabilities

In [ ]:
print("""
❌ DANGEROUS - Never do these:

1. Using eval() for calculations:
   @tool
   def bad_calc(expr): return eval(expr)  # Code injection!

2. Reading any file without validation:
   @tool
   def bad_read(path): return open(path).read()  # Can read /etc/passwd!

3. Executing arbitrary code:
   @tool
   def bad_exec(code): exec(code)  # Full system access!

4. Making requests to any URL:
   @tool  
   def bad_fetch(url): return requests.get(url).text  # SSRF attacks!

✅ ALWAYS:
- Validate and sanitize ALL inputs
- Use whitelists, not blacklists
- Limit file access to specific directories
- Block internal network access
- Set timeouts and size limits
- Log all tool usage for auditing
""")

---

## 🎉 Checkpoint

You've learned:
- ✅ What tools are and why agents need them
- ✅ How to create safe, well-documented tools
- ✅ Building a calculator without dangerous `eval()`
- ✅ File operations with proper validation
- ✅ Code execution in a sandboxed environment
- ✅ API calls with security checks

---

## ✋ Try It Yourself

Create your own custom tool! Ideas:
1. A **unit converter** (miles to km, celsius to fahrenheit)
2. A **text analyzer** (word count, character count, reading time)
3. A **password generator** (secure random passwords)

<details>
<summary>💡 Hint: Unit Converter</summary>

```python
@tool
def convert_units(value: str, from_unit: str, to_unit: str) -> str:
    """
    Convert between units.
    
    Supports: km/miles, celsius/fahrenheit, kg/pounds
    """
    conversions = {
        ('km', 'miles'): lambda x: x * 0.621371,
        ('miles', 'km'): lambda x: x * 1.60934,
        ('celsius', 'fahrenheit'): lambda x: x * 9/5 + 32,
        ('fahrenheit', 'celsius'): lambda x: (x - 32) * 5/9,
    }
    
    key = (from_unit.lower(), to_unit.lower())
    if key in conversions:
        result = conversions[key](float(value))
        return f"{value} {from_unit} = {result:.2f} {to_unit}"
    return f"Error: Unknown conversion {from_unit} -> {to_unit}"
```
</details>

In [ ]:
# Your custom tool here!

# @tool
# def my_custom_tool(...) -> str:
#     """
#     Description of what your tool does.
#     
#     Args:
#         ...
#     
#     Returns:
#         ...
#     """
#     pass

---

## 🚀 Challenge (Optional)

Build a **Text Analyzer Tool** that provides comprehensive statistics:

- Character count (with and without spaces)
- Word count
- Sentence count
- Paragraph count
- Average word length
- Estimated reading time (assuming 200 words/minute)
- Most common words (top 5)

This will be useful when we build agents that process documents!

---

## 📖 Further Reading

- [LangChain Tools Documentation](https://python.langchain.com/docs/modules/tools/)
- [Building Custom Tools](https://python.langchain.com/docs/how_to/custom_tools/)
- [Tool Use Safety Best Practices](https://docs.anthropic.com/claude/docs/tool-use)
- [Python AST Module](https://docs.python.org/3/library/ast.html)

---

## 🧹 Cleanup

In [ ]:
# Cleanup for DGX Spark
import gc

# Clear GPU memory if available
try:
    import torch
    if torch.cuda.is_available():
        torch.cuda.synchronize()
        torch.cuda.empty_cache()
        allocated = torch.cuda.memory_allocated() / 1e9
        print(f"✅ GPU memory cleared ({allocated:.2f} GB still allocated)")
except ImportError:
    pass

# Python garbage collection
gc.collect()
print("✅ Cleanup complete!")

---

## 🎓 Summary

In this lab, you built **5+ production-ready tools**:

| Tool | Purpose | Security Measures |
|------|---------|-------------------|
| `calculate` | Safe math evaluation | AST parsing (no `eval()`) |
| `web_search` | Information lookup | Simulated (use real API in prod) |
| `execute_python` | Code execution | Pattern blocking, restricted globals |
| `read_file` | File access | Extension whitelist, size limits |
| `call_api` | HTTP requests | URL validation, block internal IPs |
| `get_current_datetime` | Time lookup | Safe by design |
| `json_parser` | JSON extraction | Error handling |

**Key Principles:**
1. Always validate and sanitize inputs
2. Provide clear, detailed tool descriptions
3. Handle errors gracefully
4. Use whitelists, not blacklists
5. Set appropriate limits (size, time, access)

**Next up:** Lab 3.6.2 - Building ReAct Agents with Tool Selection